In [ ]:
import os
import cv2
import numpy as np
import torch
from torch.autograd import Variable
from torchvision.transforms import ToTensor
from unet import UNet

# Set up input directory, output directory, and slide file
input_dir = 'data/Lauren_PreEclampsia_Raw_Images'
output_dir = 'output'
slide_file = 'T20/Image0.tif'

# Read the kidney H&E slide
slide_path = os.path.join(input_dir, slide_file)
input_image = cv2.imread(slide_path, cv2.IMREAD_COLOR)

# Resize the input image to match the model's input size
input_image_resized = cv2.resize(input_image, (512, 512))

# Normalize the input image
input_image_normalized = input_image_resized / 255.0

# Convert the input image to a PyTorch tensor
input_tensor = ToTensor()(input_image_normalized).unsqueeze(0)

# Load the pretrained model
model = UNet()
# 'Omni-Seg/Omni_seg_pipeline_gpu/snapshots_2D/fold1_with_white_Omni-Seg_normalwhole_1201/MOTS_DynConv_fold1_with_white_Omni-Seg_normalwhole_1201_e100.pth.pth'
# 'Omni-Seg/Omni_seg_pipeline_gpu/snapshots_2D/fold1_with_white_Omni-Seg_normalwhole_1201/MOTS_DynConv_fold1_with_white_UNet2D_ns_normalwhole_1106_e89.pth'
model_weights_path = 'Omni-Seg/Omni_seg_pipeline_gpu/snapshots_2D/fold1_with_white_Omni-Seg_normalwhole_1201/MOTS_DynConv_fold1_with_white_scale_psuedo_allMatching_withsemi_0.1_0.1_normalwhole_0217_e81.pth'
model.load_state_dict(torch.load(model_weights_path))
model.eval()

# Perform glomeruli segmentation
with torch.no_grad():
    input_variable = Variable(input_tensor)
    output_variable = model(input_variable)
    output_mask = torch.argmax(output_variable, dim=1).squeeze().numpy()

# Save the output segmentation mask to the output directory
output_path = os.path.join(output_dir, f'{os.path.splitext(slide_file)[0]}_segmentation_mask.npy')
np.save(output_path, output_mask)
